# 서울스토어 크롤러2

분류용 상품 이미지를 크롤링하기 위한 메타 정보 크롤러입니다. a.k.a "통" 만들기 사전작업

서울스토어의 카테고리별 상품 정보(이미지, 상세정보 링크)를 크롤링하는 크롤러입니다. 크롤링 결과를 활용해서 상품 메타정보를 저장 + 상품 이미지(별도의 크롤러 활용)를 저장합니다.

크롤링한 카테고리는 아래와 같습니다.
- 상의
- 하의
- 원피스

크롤링 결과는 {"key": 고유번호(5자리수), "product_url": 상품url, "img_url": 이미지url} 형태의 json 파일로 저장합니다.

In [2]:
from __future__ import unicode_literals
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import glob
import json
import os
import re
import sys
import time
import traceback
from builtins import open
from time import sleep

In [3]:
#카테고리별 url(상의, 하의, 원피스)
root_urls = ['https://www.seoulstore.com/categories/1251/regDatetime/desc',
            'https://www.seoulstore.com/categories/1253/regDatetime/desc',
            'https://www.seoulstore.com/categories/1252/regDatetime/desc']

In [16]:
def get_products(url, num, filepath, wait_time):
    
    """
    Arguments:
    - url: str. 크롤링할 url
    - num: int. url당 크롤링할 아이템 수
    - filepath: str. 크롤링 결과를 저장할 json 파일 경로
    - wait_time: int. 로딩을 몇 초까지 기다릴지
    
    Return:
    - 없음
    
    크롤링 완료 후 "key":고유번호(5자리수)를 추가해줄 것임 (key를 사후에 추가하는 이유는 중복 크롤링 처리를 쉽게 하기 위함임)
    """
    product_set = set()   #중복 크롤링 거르기 위한 셋. product_url을 원소로 함
    
    browser = webdriver.Chrome('chromedriver')   #크롬 브라우저 실행
    wait = WebDriverWait(browser, wait_time)
        
    browser.get(url)
    body = browser.find_element_by_tag_name('body')

    count = 0    #더 이상 로드되는 데이터가 없을 시 크롤링 종료하기 위해 필요한 count임
    prev_posts_count = 0
    ele_posts = browser.find_elements_by_class_name('image_container')
    
    ##########추가한 부분###########
    while len(ele_posts) < num:
        body.send_keys(Keys.PAGE_DOWN)
        ele_posts = browser.find_elements_by_class_name('image_container')
        if len(ele_posts) % 100 == 0:
            print(len(ele_posts))
        
        cur_posts_count = len(ele_posts)
        if prev_posts_count == cur_posts_count:
            count += 1
        else: count = 0
        if count > 50:
                break
                
        prev_posts_count = cur_posts_count
            
    ##########추가한 부분 끝##########
    
    for ele in ele_posts:
        product_url= ele.find_element_by_tag_name('a').get_attribute('href')
        if product_url not in product_set:
            dict_post = { "product_url": product_url }
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ImageLoader.ratio_1_1.loaded')))
            ele_img = ele.find_element_by_class_name('ImageLoader.ratio_1_1.loaded')
            dict_post["img_url"] = ele_img.get_attribute("src")
            product_set.add(product_url)
                    
            out = json.dumps(dict_post, ensure_ascii=False)    #json 형식으로 정보 변환
            out += ', '    #아이템 정보 분류하기 위해 끝에 쉼표 추고
            with open(filepath, "a", encoding="utf-8") as f:
                f.write(out)
    
    browser.close()

In [6]:
filepath = '../tong.json'

if not filepath:     #파일 없을 경우 만들어주기
    open(filepath, 'a').close()
        
for url in root_urls:    #크롤링
    get_products(url, 3000, '../tong.json', 300)

0
0
300
300
300
300
300
300
300
300
300
300
300
300
300
600
600
600
600
600
600
600
600
600
900
900
900
900
900
900
1200
1200
1200
1200
1500
1500
1500
1500
1800
1800
1800
1800
2100
2100
2100
2400
2400
2400
2700
2700
2700
3000
0
300
300
300
300
300
300
300
300
300
300
300
300
600
600
600
600
600
600
900
900
900
900
1200
1200
1200
1200
1500
1500
1500
1500
1800
1800
1800
1800
2100
2100
2100
2400
2400
2400
2700
2700
2700
3000
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
300
600
600
600
600
600
600
900
900
900
900
1200
1200
1200
1200
1200
1500
1500
1500
1500
1800
1800
1800
2100
2100
2100
2100
2400
2400
2400
2400


KeyboardInterrupt: 

In [17]:
get_products(root_urls[2], 3000, '../tong.json', 300)

0
0
0
0
0
300
300
300
300
300
300
300
300
300
300
600
600
600
600
600
900
900
900
900
900
900
900
900
900
900
900
1200
1200
1200
1200
1500
1500
1500
1500
1800
1800
1800
1800
2100
2100
2100
2100
2400
2400
2400


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".ImageLoader.ratio_1_1.loaded"}
  (Session info: chrome=81.0.4044.138)


In [30]:
with open('../tong.json', 'r', encoding = 'utf-8') as f:
    file = f.read()

len(file.split('},'))

7088

In [33]:
file.split('},')[7086]

' {"product_url": "https://www.seoulstore.com/products/1137926/detail", "img_url": "https://images.seoulstore.com/products/43f8a4a2b79e4132c99bd357f71a9748.jpg?d=640xauto"'

In [29]:
url = root_urls[2]
num = 3000
path = '../tong.json'
wait_time = 300

product_set = set()   #중복 크롤링 거르기 위한 셋. product_url을 원소로 함
browser = webdriver.Chrome('chromedriver')   #크롬 브라우저 실행
wait = WebDriverWait(browser, wait_time)
        
browser.get(url)
body = browser.find_element_by_tag_name('body')

count = 0    #더 이상 로드되는 데이터가 없을 시 크롤링 종료하기 위해 필요한 count임
prev_posts_count = 0
ele_posts = browser.find_elements_by_class_name('image_container')
    
##########추가한 부분###########
while len(ele_posts) < num:
    body.send_keys(Keys.PAGE_DOWN)
    ele_posts = browser.find_elements_by_class_name('image_container')
    if len(ele_posts) % 100 == 0:
        print(len(ele_posts))
        
    cur_posts_count = len(ele_posts)
    if prev_posts_count == cur_posts_count:
        count += 1
    else: count = 0
    if count > 50:
        break
                
    prev_posts_count = cur_posts_count
            
##########추가한 부분 끝##########
for ele in ele_posts[1076:]:
    product_url= ele.find_element_by_tag_name('a').get_attribute('href')
    if product_url not in product_set:
        dict_post = { "product_url": product_url }
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ImageLoader.ratio_1_1.loaded')))
        ele_img = ele.find_element_by_class_name('ImageLoader.ratio_1_1.loaded')
        dict_post["img_url"] = ele_img.get_attribute("src")
        product_set.add(product_url)
                    
        out = json.dumps(dict_post, ensure_ascii=False)    #json 형식으로 정보 변환
        out += ', '    #아이템 정보 분류하기 위해 끝에 쉼표 추고
        with open(filepath, "a", encoding="utf-8") as f:
            f.write(out)
browser.close()

300
300
300
300
300
300
300
300
300
300
300
300
300
600
600
600
600
600
600
600
900
900
900
900
900
1200
1200
1200
1200
1500
1500
1500
1500
1800
1800
1800
1800
1800
2100
2100
2100
2400
2400
2400


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".ImageLoader.ratio_1_1.loaded"}
  (Session info: chrome=81.0.4044.138)


In [35]:
for ele in ele_posts[1090:]:
    product_url= ele.find_element_by_tag_name('a').get_attribute('href')
    if product_url not in product_set:
        dict_post = { "product_url": product_url }
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'ImageLoader.ratio_1_1.loaded')))
        ele_img = ele.find_element_by_class_name('ImageLoader.ratio_1_1.loaded')
        dict_post["img_url"] = ele_img.get_attribute("src")
        product_set.add(product_url)
                    
        out = json.dumps(dict_post, ensure_ascii=False)    #json 형식으로 정보 변환
        out += ', '    #아이템 정보 분류하기 위해 끝에 쉼표 추고
        with open(filepath, "a", encoding="utf-8") as f:
            f.write(out)

In [36]:
with open('../tong.json', 'r', encoding = 'utf-8') as f:
    file = f.read()

len(file.split('},'))

8691